In [60]:
import numpy as np
import pandas as pd

df = pd.read_csv('data/education/사설학원수.csv')
# df.head(), df.shape, df['지역별'].unique()

In [61]:
# '시점'을 datetime 형식으로 변환
df['시점'] = pd.to_datetime(df['시점'].astype(str) + '-01-01')

# 인덱스로 설정
yearly_data = df.set_index('시점')

# 지역별로 그룹화 후 리샘플링
monthly_dfs = []

for region, group in yearly_data.groupby('지역별'):
    monthly_df = group.resample('ME').agg({'사설학원수': 'mean'})

    # 보간
    monthly_df = monthly_df.interpolate(method='linear')
    
    # NaN 값 처리
    monthly_df.fillna(0, inplace=True)
    
    # 시점 다시 설정
    monthly_df.reset_index(inplace=True)
    
    # 마지막 년도에 12월 데이터 추가
    last_year = monthly_df['시점'].dt.year.max()
    last_month = monthly_df['시점'].dt.month.max()
    
    # 12월 데이터가 없으면 추가
    if last_month < 12:
        new_row = pd.DataFrame({
            '사설학원수': [monthly_df['사설학원수'].iloc[-1]],
            '시점': [pd.Timestamp(year=last_year, month=12, day=1)],
            '지역별': [region]
        })
        monthly_df = pd.concat([monthly_df, new_row], ignore_index=True)

    # 지역별 컬럼 추가
    monthly_df['지역별'] = region
    
    monthly_dfs.append(monthly_df)

# 모든 지역별 데이터를 하나의 DataFrame으로 결합
df_new = pd.concat(monthly_dfs, ignore_index=True)

# '시점'을 문자열로 다시 포맷
df_new['시점'] = df_new['시점'].dt.strftime('%Y.%m')


# 결과 확인
print(df_new.head())
print(df_new.shape)
print(df_new['시점'].unique())

df_new = df_new.sort_values(by=['지역별', '시점'])
df_new['사설학원수'] = df_new['사설학원수'].map(lambda x: f"{x:.3f}")

# 결과 확인
print("정렬 후 데이터:")
print(df_new.head())
print(f"데이터 형태: {df_new.shape}")
print("고유한 시점 값:")
print(df_new['시점'].unique())

# 데이터 저장
df_new.to_csv('data/education/사설학원수(월별).csv', index=False)

# # 결과 확인
# print(final_monthly_df.tail(12), final_monthly_df.shape, final_monthly_df['지역별'].unique())

# # 데이터 저장
# final_monthly_df.to_csv('data/education/사설학원수(월별).csv', index=False)


        시점        사설학원수 지역별
0  2003.01  2146.000000  강원
1  2003.02  2134.833333  강원
2  2003.03  2123.666667  강원
3  2003.04  2112.500000  강원
4  2003.05  2101.333333  강원
(3989, 3)
['2003.01' '2003.02' '2003.03' '2003.04' '2003.05' '2003.06' '2003.07'
 '2003.08' '2003.09' '2003.10' '2003.11' '2003.12' '2004.01' '2004.02'
 '2004.03' '2004.04' '2004.05' '2004.06' '2004.07' '2004.08' '2004.09'
 '2004.10' '2004.11' '2004.12' '2005.01' '2005.02' '2005.03' '2005.04'
 '2005.05' '2005.06' '2005.07' '2005.08' '2005.09' '2005.10' '2005.11'
 '2005.12' '2006.01' '2006.02' '2006.03' '2006.04' '2006.05' '2006.06'
 '2006.07' '2006.08' '2006.09' '2006.10' '2006.11' '2006.12' '2007.01'
 '2007.02' '2007.03' '2007.04' '2007.05' '2007.06' '2007.07' '2007.08'
 '2007.09' '2007.10' '2007.11' '2007.12' '2008.01' '2008.02' '2008.03'
 '2008.04' '2008.05' '2008.06' '2008.07' '2008.08' '2008.09' '2008.10'
 '2008.11' '2008.12' '2009.01' '2009.02' '2009.03' '2009.04' '2009.05'
 '2009.06' '2009.07' '2009.08' '2009.09' 